In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Reading data

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_submit = pd.read_csv('data/simple_submission.csv')

C:\Users\Ut Luom\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Ut Luom\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
c_train = df_train.copy()
c_test = df_test.copy()

c_train['train'] = 1
c_test['train'] = 0
index = df_test['id']
df = pd.concat([c_train, c_test], axis=0, sort=False)

# 2. pre-processing

In [5]:
time_field = [
            'Field_1', 'Field_2', 'Field_3', 'Field_5', 'Field_6', 'Field_7','Field_8',
            'Field_9', 'Field_11', 'Field_15', 'Field_25', 'Field_32', 'Field_33',
            'Field_35', 'Field_40', 'Field_43', 'Field_44', 'ngaySinh',
            'F_startDate', 'F_endDate', 'E_startDate', 'E_endDate', 'C_startDate',
            'C_endDate', 'G_startDate', 'G_endDate', 'A_startDate', 'A_endDate'
]

cat_field = [
            'Field_4',
            'Field_18', 'Field_12', 'Field_34', 'Field_36', 'Field_38', 'Field_45', 'Field_46',
            'Field_47', 'Field_48', 'Field_49', 'Field_54', 'Field_55', 'Field_56', 'Field_61', 'Field_62',
            'Field_65', 'Field_66', 'Field_68', 'gioiTinh', 'diaChi', 'maCv', 
            'info_social_sex', 'data.basic_info.locale', 'currentLocationCity',
            'currentLocationCountry', 'currentLocationName', 'currentLocationState',
            'homeTownCity', 'homeTownCountry', 'homeTownName', 'homeTownState', 'brief'
]

num_field = [col for col in df.columns if col not in time_field+cat_field]
print(num_field)

['id', 'label', 'Field_10', 'Field_13', 'Field_14', 'Field_16', 'Field_17', 'Field_19', 'Field_20', 'Field_21', 'Field_22', 'Field_23', 'Field_24', 'Field_26', 'Field_27', 'Field_28', 'Field_29', 'Field_30', 'Field_31', 'namSinh', 'Field_37', 'Field_39', 'Field_41', 'Field_42', 'Field_50', 'Field_51', 'Field_52', 'Field_53', 'Field_57', 'Field_58', 'Field_59', 'Field_60', 'Field_63', 'Field_64', 'Field_67', 'Field_69', 'Field_70', 'Field_71', 'Field_72', 'Field_73', 'Field_74', 'Field_75', 'Field_76', 'Field_77', 'friendCount', 'subscriberCount', 'currentLocationLocationId', 'currentLocationLatitude', 'currentLocationLongitude', 'homeTownLocationId', 'homeTownLatitude', 'homeTownLongitude', 'topFriends', 'numOrg', 'F_numOrg', 'F_numQuery', 'E_numOrg', 'E_numQuery', 'C_numOrg', 'C_numQuery', 'G_numOrg', 'G_numQuery', 'A_numOrg', 'A_numQuery', 'summary_6m', 'summary_3m', 'summary_1m', 'summary_1w', 'partner0_A', 'partner0_B', 'partner0_C', 'partner0_D', 'partner0_E', 'partner0_F', 'partn

In [6]:
def get_unknown_category(df_train, df_test, cat=cat_field):
    result = pd.DataFrame(columns=['Feature Name', 'Train Unique', 'Test Unique', 'Unknown Value'])
    train_unique = df_train[cat].stack().reset_index(level=0, drop=True)
    train_unique = train_unique.groupby(level=0).unique().reindex(index=df_train[cat].columns)
    test_unique = df_test[cat].stack().reset_index(level=0, drop=True)
    test_unique = test_unique.groupby(level=0).unique().reindex(index=df_test[cat].columns)
    for index, feat in enumerate(cat):
        a = test_unique.loc[feat]
        u1 = train_unique.loc[feat]
        u2 = test_unique.loc[feat]
        unknown = np.where(~np.in1d(u2, u2))[0]
        result.loc[index] = [feat, len(u1), len(u2), len(unknown)]
    return result

In [7]:
result = get_unknown_category(df_train.astype(str), df_test.astype(str))
result

,Feature Name,Train Unique,Test Unique,Unknown Value
0,Field_4,3,3,0
1,Field_18,6588,2660,0
2,Field_12,8,8,0
3,Field_34,2636,1908,0
4,Field_36,35,30,0
5,Field_38,7,6,0
6,Field_45,10958,5439,0
7,Field_46,14312,6553,0
8,Field_47,6,6,0
9,Field_48,12651,5901,0


In [8]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,73411,NaN,NaN,NaN,36705,21192.1,0,18352.5,36705,55057.5,73410
label,53030,NaN,NaN,NaN,0.316726,0.465204,0,0,0,1,1
Field_1,34624,28806,2017-03-24T20:10:37.62Z,343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Field_2,34624,24268,2017-03-24T20:10:37.62Z,338,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Field_3,34624,NaN,NaN,NaN,1.53596,0.498713,1,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...
Field_79,38533,NaN,NaN,NaN,34.6264,40.1541,0,5.65685,17.0392,52.3068,359.917
Field_80,49880,NaN,NaN,NaN,-4.83718,36.4411,-303,-8.33929,1.55,8.5,238
Field_81,49450,NaN,NaN,NaN,-39.8967,70.9605,-332,-89,0,1,238
Field_82,73411,NaN,NaN,NaN,1.6965,1.09638,1,1,1,2,4


## 2.1. Missing Values

In [9]:
def missing_value(df):
    mis_val = df.isnull().sum()
    mis_val_per = 100 * df.isnull().sum() / len(df)
    mis_val_tab = pd.concat([mis_val, mis_val_per], axis=1)
    mis_val_re  = mis_val_tab.rename(columns={
        0: 'Miss',
        1: 'Per'
    })
    mis_val_re = mis_val_re[mis_val_re.iloc[:, 1] !=0].sort_values('Per', ascending=False).round(1)
    print("Có tất cả " + str(df.shape[1]) + " cột.\n"
          "Trog đó có " + str(mis_val_re.shape[0]) + " cột bị thiếu data")
    return mis_val_re

In [10]:
missing = missing_value(df)
missing

Có tất cả 196 cột.
Trog đó có 193 cột bị thiếu data


,Miss,Per
Field_35,64202,87.5
Field_12,63489,86.5
Field_11,63438,86.4
Field_18,62005,84.5
maCv,61029,83.1
...,...,...
partner1_L,7395,10.1
partner1_K,7395,10.1
partner3_A,7395,10.1
num_of_phone,3200,4.4


In [11]:
print(df['label'].value_counts())

0.0    36234
1.0    16796
Name: label, dtype: int64


In [12]:
def infomation(df):
    x, y = df
    if x != x and y != y:
        return 'nan'
    if x != x:
        return y.lower()
    return x.lower()

df["sex"] = df[['gioiTinh', 'info_social_sex']].apply(infomation, axis=1).astype('object')

drop = ['Field_14', 'Field_16', 'Field_17', 'Field_24', 'Field_26', 'Field_30', 'Field_31', 'Field_37', 'Field_52',
        'Field_57', 'partner0_B', 'partner0_K', 'partner0_L', 'partner1_B', 'partner1_D', 'partner1_E', 'partner1_F',
        'partner1_K', 'partner1_L', 'partner2_B', 'partner2_G', 'partner2_K', 'partner2_L', 'partner3_B', 'partner3_F',
        'partner3_G', 'partner3_H', 'partner3_K', 'partner3_L', 'partner4_A', 'partner4_B', 'partner4_C', 'partner4_D',
        'partner4_E', 'partner4_F', 'partner4_G', 'partner4_H', 'partner4_K', 'partner4_L', 'partner5_B', 'partner5_C',
        'partner5_H', 'partner5_K', 'partner5_L', 'Field_34', 'gioiTinh', 'info_social_sex', 'data.basic_info.locale', 'homeTownCountry',
        'Field_2', 'Field_5', 'Field_49', 'Field_9', 'ngaySinh', 'namSinh', 'Field_10', 'Field_13', 'Field_21', 
        'Field_22', 'Field_38','Field_41', 'Field_42','Field_50', 'Field_53', 'Field_62', 'currentLocationLongitude', 'homeTownLongitude',
        'E_numQuery', 'C_numOrg', 'C_numQuery', 'G_numQuery', 'A_numQuery', 'summary_6m',
        'summary_3m', 'summary_1m', 'summary_1w', 'partner0_C', 'partner0_D', 'partner0_E',
        'partner0_F', 'partner0_H', 'partner1_C', 'partner1_G', 'partner1_H', 'partner2_A',
        'partner2_C', 'partner2_D', 'partner2_E', 'partner2_H', 'partner3_C', 'partner3_E',
        'partner5_D', 'partner5_E', 'partner5_F', 'partner5_G', 'num_of_phone',
        'Field_51', 'Field_73','partner2_F', 'Field_35', 'Field_11', 'Field_15',
        'Field_1', 'Field_6', 'Field_7', 'Field_8', 'Field_25', 
        'Field_33', 'Field_40', 'Field_43', 'Field_44', 'Field_18',
        'F_startDate', 'F_endDate', 'E_startDate', 'E_endDate', 'C_startDate', 'C_endDate', 'G_startDate',
        'G_endDate', 'A_startDate', 'A_endDate','Field_4', 'Field_12', 'Field_45', 'Field_36', 'Field_54', 'Field_55', 'Field_66', 
        'Field_68', 'currentLocationState', 'homeTownCity', 'homeTownName', 'homeTownState', 'Field_32']

df = df.drop(drop, axis=1)

In [13]:
df.shape

(73411, 62)

In [14]:
for col_name in df.columns:
    if df[col_name].dtypes == 'object':
        unique_cat = len(df[col_name].unique())
        print("Trong cột " + str(col_name) + " có " + str(unique_cat) + " biến object")

Trong cột diaChi có 27702 biến object
Trong cột Field_46 có 17903 biến object
Trong cột Field_47 có 6 biến object
Trong cột Field_48 có 15761 biến object
Trong cột Field_56 có 1516 biến object
Trong cột Field_61 có 37 biến object
Trong cột Field_65 có 11 biến object
Trong cột maCv có 4547 biến object
Trong cột currentLocationCity có 1021 biến object
Trong cột currentLocationCountry có 52 biến object
Trong cột currentLocationName có 1276 biến object
Trong cột brief có 21 biến object
Trong cột sex có 3 biến object


In [15]:
df = df.drop(columns=['diaChi', 'Field_46', 'Field_48', 'Field_56', 'currentLocationCity', 'maCv', 'currentLocationName'], axis=1)


In [16]:
df.shape

(73411, 55)

In [17]:
for col_name in df.columns:
    if df[col_name].dtypes == 'object':
        unique_cat = len(df[col_name].unique())
        print("Trong cột " + str(col_name) + " có " + str(unique_cat) + " biến object")

Trong cột Field_47 có 6 biến object
Trong cột Field_61 có 37 biến object
Trong cột Field_65 có 11 biến object
Trong cột currentLocationCountry có 52 biến object
Trong cột brief có 21 biến object
Trong cột sex có 3 biến object


In [18]:
todummy_list = ['Field_47', 'Field_61', 'Field_65', 'currentLocationCountry', 'brief', 'sex']

In [19]:
def dummy_df(df, todummy_df):
    for i in todummy_list:
        dummies = pd.get_dummies(df[i], prefix=i, dummy_na=False)
        df = df.drop(i, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [20]:
df = dummy_df(df, todummy_list)

In [21]:
print(df.head(10))

   id  label  Field_3  Field_19   Field_20  Field_23   Field_27  Field_28  \
0   0    1.0      1.0       0.0  4258600.0      16.0        0.0       0.0   
1   1    0.0      NaN       NaN        NaN       NaN        NaN       NaN   
2   2    0.0      2.0       0.0  5000000.0      10.0        0.0       0.0   
3   3    0.0      NaN       NaN        NaN       NaN        NaN       NaN   
4   4    1.0      NaN       NaN        NaN       NaN        NaN       NaN   
5   5    0.0      NaN       NaN        NaN       NaN        NaN       NaN   
6   6    0.0      2.0       0.0  3970000.0      33.0        0.0       0.0   
7   7    1.0      2.0       0.0  1043000.0      30.0        0.0       0.0   
8   8    0.0      1.0       0.0  1210000.0       0.0  1210000.0   54450.0   
9   9    0.0      NaN       NaN        NaN       NaN        NaN       NaN   

   Field_29  Field_39  ...  brief_7  brief_7cy  brief_8  brief_cb1  brief_cb2  \
0       0.0       1.0  ...        0          0        0          1     

## 2.2 Handling Mising Values

In [22]:
miss = missing_value(df)
miss

Có tất cả 174 cột.
Trog đó có 46 cột bị thiếu data


,Miss,Per
Field_74,44993,61.3
topFriends,44841,61.1
Field_75,41055,55.9
Field_76,41055,55.9
Field_77,41055,55.9
Field_64,40681,55.4
friendCount,40229,54.8
currentLocationLocationId,40229,54.8
subscriberCount,40229,54.8
currentLocationLatitude,40229,54.8


In [23]:
!pip install -U scikit-learn

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.1
    Uninstalling scikit-learn-0.23.1:
      Successfully uninstalled scikit-learn-0.23.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 145] The directory is not empty: 'c:\\users\\ut luom\\anaconda3\\lib\\site-packages\\~-learn\\mixture\\tests'



In [24]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='median')
imp.fit(df)
df = pd.DataFrame(data=imp.transform(df), columns=df.columns)

In [25]:
missing = missing_value(df)
missing

Có tất cả 174 cột.
Trog đó có 0 cột bị thiếu data


,Miss,Per


In [26]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    # Get feature names
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    # Find interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    # Remove interaction terms with all 0 values            
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df

In [27]:
df = add_interactions(df)
print(df.head(5))

MemoryError: Unable to allocate 8.33 GiB for an array with shape (73411, 15225) and data type float64